In [2]:
pip install opencv-python torch ultralytics scikit-learn numpy



  Using cached ultralytics-8.3.165-py3-none-any.whl.metadata (37 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 5.5 MB/s eta 0:00:00


In [4]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from utils import extract_features, match_players, draw_player_boxes

# Load model
model = YOLO('weights/best.pt')  # Replace with actual model name if different

# Load videos
broadcast_path = 'data/broadcast.mp4'
tacticam_path = 'data/tacticam.mp4'

cap_broadcast = cv2.VideoCapture(broadcast_path)
cap_tacticam = cv2.VideoCapture(tacticam_path)

player_feats_broadcast = {}
player_feats_tacticam = {}

print("\n[INFO] Processing Broadcast Video...")
frame_id = 0
while True:
    ret, frame = cap_broadcast.read()
    if not ret or frame_id > 100:
        break

    results = model(frame)[0]  # assume single model inference
    for box in results.boxes:
        cls = int(box.cls[0])
        if cls != 0:  # Assuming class 0 = player
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = frame[y1:y2, x1:x2]
        feat = extract_features(crop)
        player_feats_broadcast[f"b_{frame_id}_{x1}"] = feat
    frame_id += 1

print("[INFO] Processing Tacticam Video...")
cap_tacticam.set(cv2.CAP_PROP_POS_FRAMES, 0)
frame_id = 0
while True:
    ret, frame = cap_tacticam.read()
    if not ret or frame_id > 100:
        break

    results = model(frame)[0]
    for box in results.boxes:
        cls = int(box.cls[0])
        if cls != 0:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = frame[y1:y2, x1:x2]
        feat = extract_features(crop)
        player_feats_tacticam[f"t_{frame_id}_{x1}"] = feat
    frame_id += 1

# Match players
print("[INFO] Matching players across cameras...")
matches = match_players(player_feats_broadcast, player_feats_tacticam)

# Output
print("\nMatched Players:")
for b_id, t_id in matches.items():
    print(f"Broadcast ID: {b_id} <--> Tacticam ID: {t_id}")

cap_broadcast.release()
cap_tacticam.release()
cv2.destroyAllWindows()


[INFO] Processing Broadcast Video...

0: 384x640 3 players, 598.3ms
Speed: 10.0ms preprocess, 598.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 544.1ms
Speed: 2.6ms preprocess, 544.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 534.6ms
Speed: 1.7ms preprocess, 534.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 608.3ms
Speed: 4.4ms preprocess, 608.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 513.1ms
Speed: 2.3ms preprocess, 513.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 506.3ms
Speed: 1.9ms preprocess, 506.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 551.8ms
Speed: 2.0ms preprocess, 551.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 